In [1]:
from SAETrainer import SAETrainer
import wandb
import os

api = wandb.Api()

def is_available(node):
    component_name, layer, head = node.component_name, node.layer, node.head
    project_name = "sae-all-ioi-heads-low-l0"
    entity = "mega-alignment"    
    runs = api.runs(path=f"{entity}/{project_name}")
    for run in runs:
        run_component = run.name.split('--')[0]
        run_layer = int(run.name.split('--')[1].split('h')[0][1:])
        run_head = int(run.name.split('--')[1].split('h')[1].split('-')[0])
        if run_component == component_name and run_layer == layer and run_head == head:
            return True
    return False


project_name = "sae-all-ioi-heads-low-l0"
entity = "mega-alignment"
    
# run = api.run("mega-alignment/sae/byniiz7n")
runs = api.runs(path=f"{entity}/{project_name}")

def load_sae(layer, head, component):
    project_name = "sae-all-ioi-heads-low-l0"
    entity = "mega-alignment"
    
    name_mover_sae = { }  # fill this if you wanna use different runs than the default ones
    # e.g. ('z', 9, 9): 'z--l9h9-l1_0.03-819.2M',
    if (component, layer, head) in name_mover_sae.keys():
        project_name = 'sae'
        run_name = name_mover_sae[(component, layer, head)]
        version = 'v1'
    else:
        run_name = f"{component}--l{layer}h{head}-l1_0.006-250.001408M"
        version = 'v25'
        project_name = "sae-all-ioi-heads-low-l0"
        run_name = f"{component}--l{layer}h{head}-l1_0.006-250.001408M"

    runs = api.runs(path=f"{entity}/{project_name}")

    selected_run = None
    for run in runs:
        if run.name == run_name or run.id == run_name:  # Adjust condition based on your criteria
            selected_run = run
            break
    
    if selected_run is None:
        print(f"Run {run_name} not found. Layer: {layer}, head: {head}, component: {component}")
        raise ValueError("Run not found.")
    
    artifact_name = f'model-{selected_run.id}:{version}'
    
    # the runs were copied to a new project but the artifacts are still in the old project
    project_name = 'serimats'
    entity = 'georglange'
    
    artifact = api.artifact(type="model", name=f"{entity}/{project_name}/{artifact_name}")    
    if artifact is None:
        raise ValueError("Artifact not found.")
    artifact_dir = artifact.download()
    
    cfg = selected_run.config
    
    encoder = SAETrainer.load_from_checkpoint(os.path.join(artifact_dir, 'model.ckpt'), cfg, ).sae
    return encoder

In [2]:
import sys
sys.path.append('/tmp/pycharm_project_349/ioi_subspaces')
from new_codebase import Circuit

c = Circuit()
circuit_nodes = list(c.nodes.keys())

could not find model


In [6]:
c.nm[0]

Head(L9H9)

In [7]:
c.prompts

[]